## TReNDS Neuroimaging

_Multiscanner normative age and assessments prediction with brain function, structure, and connectivity_

In this challenge, participants will predict age and assessment values from two domains using features derived from brain MRI images as inputs.

Models are expected to generalize on data from a different scanner/site (site 2). All subjects from site 2 were assigned to the test set, so their scores are not available. While there are fewer site 2 subjects than site 1 subjects in the test set, the total number of subjects from site 2 will not be revealed until after the end of the competition. To make it more interesting, the IDs of some site 2 subjects have been revealed below. Use this to inform your models about site effects. Site effects are a form of bias. To generalize well, models should learn features that are not related to or driven by site effects.

The .mat files for this competition can be read in python using h5py, and the .nii file can be read in python using nilearn.


### Problem Understanding

Neuroimaging specialists look for measurable markers of behavior, health, or disorder to help identify relevant brain regions and their contribution to typical or symptomatic effects.
<br />
An fMRI scan is a functional magnetic resonance imaging scan that measures and maps the brain’s activity by the bloodflow during scanning.

Separate, unrelated large imaging dataset was utilized to learn feature templates. Then, these templates were "projected" onto the original imaging data of each subject used for this competition using spatially constrained independent component analysis (scICA) via group information guided ICA (GIG-ICA).

Source-based morphometry (SBM) loadings are subject-level weights from a group-level ICA decomposition of gray matter concentration maps from structural MRI (sMRI) scans.

Functional network connectivity (FNC) matrices are the subject-level cross-correlation values among 53 component timecourses estimated from GIG-ICA of resting state functional MRI (fMRI).

 Component spatial maps (SM). These are the subject-level 3D images of 53 spatial networks estimated from GIG-ICA of resting state functional MRI (fMRI).

### Preliminaries & Data Loading

#### Install Rapids for much faster acceleration of many data-science and machine learning tasks on GPUs

In [ ]:
import sys
!cp ../input/rapids/rapids.0.13.0 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
sys.path = ["/opt/conda/envs/rapids/lib/python3.6/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.6"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

In [ ]:
import numpy as np
import pandas as pd
import h5py

import matplotlib.pyplot as plt
import seaborn as sns
#import joypy
from nilearn import plotting
from nilearn import image

from random import seed
import warnings

import cudf
from cuml.neighbors import KNeighborsRegressor
from cuml.svm import SVR

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

seed(42)
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
loading_data = pd.read_csv('../input/trends-assessment-prediction/loading.csv')
train_scores = pd.read_csv('../input/trends-assessment-prediction/train_scores.csv')
sample_submission = pd.read_csv('../input/trends-assessment-prediction/sample_submission.csv')
fnc = pd.read_csv('../input/trends-assessment-prediction/fnc.csv')
icn_numbers = pd.read_csv('../input/trends-assessment-prediction/ICN_numbers.csv')
id_site = pd.read_csv('../input/trends-assessment-prediction/reveal_ID_site2.csv')

### Exploratory data analysis

* loading.csv - sMRI SBM loadings for both train and test samples

* IC_01 - Cerebellum
* IC_07 - Precuneus+PCC
* IC_05 - Calcarine
* IC_16 - Middle Occipital?
* IC_26 - Inf+Mid Frontal
* IC_06 - Calcarine
* IC_10 - MTG
* IC_09 - IPL+AG
* IC_18 - Cerebellum
* IC_04 - Cerebellum
* IC_12 - SMA
* IC_24 - IPL+Postcentral
* IC_15 - STG
* IC_13 - Temporal Pole
* IC_17 - Cerebellum
* IC_02 - ACC+mpfc
* IC_08 - Frontal
* IC_03 - Caudate
* IC_21 - Temporal Pole + Cerebellum
* IC_28 - Calcarine
* IC_11 - Frontal
* IC_20 - MCC
* IC_30 - Inf Frontal
* IC_22 - Insula + Caudate
* IC_29 - MTG
* IC_14 - Temporal Pole + Fusiform

In [ ]:
loading_data.head()

In [ ]:
loading_data.describe()

* train_scores.csv - age and assessment values for train samples

In [ ]:
train_scores.head()

In [ ]:
train_scores.describe()

* sample_submission.csv - a sample submission file in the correct format

In [ ]:
sample_submission.head()

* fnc.csv - static FNC correlation features for both train and test samples

* SCN - Sub-cortical Network

* ADN - Auditory Network

* SMN - Sensorimotor Network

* VSN - Visual Network

* CON - Cognitive-control Network

* DMN - Default-mode Network

* CBN - Cerebellar Network

In [ ]:
fnc.head()

In [ ]:
fnc.describe()

* reveal_ID_site2.csv - a list of subject IDs whose data was collected with a different scanner than the train samples

In [ ]:
id_site.head()

* ICN_numbers.txt - intrinsic connectivity network numbers for each fMRI spatial map; matches FNC names

In [ ]:
icn_numbers.head()

* fMRI_mask.nii - a 3D binary spatial map

In [ ]:
mask = image.load_img("../input/trends-assessment-prediction/fMRI_mask.nii")
plotting.plot_roi("../input/trends-assessment-prediction/fMRI_mask.nii")

* fMRI_train - a folder containing 53 3D spatial maps for train samples in .mat format _(only 2 samples has been taken from repo due to large size of .mat files)_

In [ ]:
def load_subject(filename, mask_niimg):
    """
    Load a subject saved in .mat format with
        the version 7.3 flag. Return the subject
        niimg, using a mask niimg as a template
        for nifti headers.
        
    Args:
        filename    <str>            the .mat filename for the subject data
        mask_niimg  niimg object     the mask niimg object used for nifti headers
    """
    
    subject_data = None
    
    with h5py.File(filename, 'r') as f:
        subject_data = f['SM_feature'][()]
    
    # It's necessary to reorient the axes, since h5py flips axis order
    subject_data = np.moveaxis(subject_data, [0,1,2,3], [3,2,1,0])
    subject_niimg = image.new_img_like(mask, subject_data, affine=mask.affine, copy_header=True)
    return subject_niimg


### Exploratory Data Analysis

#### Compoment spatial maps visualization

In [ ]:
plotting.plot_prob_atlas(load_subject('../input/trends-assessment-prediction/fMRI_train/10002.mat', mask),
                         view_type='filled_contours',
                         draw_cross=False,
                         threshold='auto')

In [ ]:
plotting.plot_prob_atlas(load_subject('../input/trends-assessment-prediction/fMRI_train/10009.mat', mask),
                         view_type='filled_contours',
                         draw_cross=False,
                         threshold='auto')

* fMRI_test - a folder containing 53 3D spatial maps for test samples in .mat format _(only 2 samples has been taken from repo due to large size of .mat files)_

In [ ]:
plotting.plot_prob_atlas(load_subject('../input/trends-assessment-prediction/fMRI_test/10003.mat', mask),
                         view_type='filled_contours',
                         draw_cross=False,
                         threshold='auto')

In [ ]:
plotting.plot_prob_atlas(load_subject('../input/trends-assessment-prediction/fMRI_test/10016.mat', mask),
                         view_type='filled_contours',
                         draw_cross=False,
                         threshold='auto')

#### Brain age distribtion plot

In [ ]:
plt.figure(figsize=(20,12))
sns.barplot(train_scores['age'].astype(int).value_counts().index,
            train_scores['age'].astype(int).value_counts().values)

plt.xlabel('Age', fontsize=12)
plt.ylabel('Number of Occurrences', fontsize=12)

#### Plot the distributions of the training_scores set

In [ ]:
fig, ax = plt.subplots(1, 5, figsize=(20, 5))

sns.distplot(train_scores['age'], ax=ax[0])
ax[0].set_title('Age')

sns.distplot(train_scores['domain1_var1'].fillna(method='ffill'), ax=ax[1])
ax[1].set_title('Domain 1 - Var 1')

sns.distplot(train_scores['domain1_var2'].fillna(method='ffill'), ax=ax[2])
ax[2].set_title('Domain 1 - Var 2')

sns.distplot(train_scores['domain2_var1'].fillna(method='ffill'), ax=ax[3])
ax[3].set_title('Domain 2 - Var 1')

sns.distplot(train_scores['domain2_var2'].fillna(method='ffill'), ax=ax[4])
ax[4].set_title('Domain 2 - Var 2')

fig.suptitle('Target distributions', fontsize=14)

The kurtosis of the training scores is small, meaning that there is not much weight in the tails.

In [ ]:
train_scores.kurtosis()

#### Check the correlation between age and features in training_scores

In [ ]:
sns.heatmap(train_scores.drop('Id', axis=1).corr(), annot = True, cmap="RdYlGn")

#### Distribution of loading gray matter concentration maps from structural MRI (sMRI) scans.

In [ ]:
#ax, fig = joypy.joyplot(loading_data.drop('Id', axis=1), figsize=(14,10))

#plt.title('Source-based morphometry loadings distribution', fontsize=22)

#### Correlation of IC concentraion maps

In [ ]:
plt.figure(figsize=(20,12))
sns.heatmap(loading_data.drop('Id', axis=1).corr(), annot = True, cmap="RdYlGn")

### Feature Engineering

#### Process id_site to loading_data and fnc dimension by index and fill if the result is produced by different scanner

In [ ]:
dataset = pd.DataFrame(fnc['Id'])
dataset['other_scanner'] = dataset["Id"].isin(id_site['Id'])
dataset['other_scanner'] = dataset['other_scanner'].astype(int)

dataset.replace(1, 0.1, inplace=True)
dataset.replace(0, -0.1, inplace=True)

#### Averaging Null values 

In [ ]:
def check_na(df):
    name =[x for x in globals() if globals()[x] is df][0]
    print(df.shape[0] - df.dropna().shape[0], "out of",  df.shape[0], "missing values in rows in", name)

check_na(train_scores)
check_na(fnc)
check_na(loading_data)
check_na(id_site)

In [ ]:
train_scores.fillna(train_scores.mean(), inplace=True)

#### Normalization of loading_data and fnc

In [ ]:
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
loading_data_normalized = min_max_scaler.fit_transform(loading_data)
fnc_normalized = min_max_scaler.fit_transform(fnc)

In [ ]:
loading_data.iloc[:,1:] = pd.DataFrame(loading_data_normalized, columns=loading_data.columns).iloc[:,1:]
fnc.iloc[:,1:] = pd.DataFrame(fnc_normalized, columns=fnc.columns).iloc[:,1:]

#### Giving less importance to FNC features since they are easier to overfit due to high dimensionality.

In [ ]:
FNC_SCALE = 1/500

In [ ]:
dataset = pd.concat([dataset, fnc.iloc[:,1:] * FNC_SCALE], axis=1)

#### Merging datasets

In [ ]:
dataset = dataset.merge(loading_data, on="Id")
dataset

### Modeling

#### Creating train and test set

In [ ]:
X_train = cudf.DataFrame.from_pandas(dataset.loc[dataset["Id"].isin(train_scores["Id"]) == True])
Y_train = cudf.DataFrame.from_pandas(train_scores)
X_test = cudf.DataFrame.from_pandas(dataset.loc[dataset["Id"].isin(train_scores["Id"]) == False])

In [ ]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)

#### Parameters playground

In [ ]:
# -- KFold
NUM_FOLDS = 6

# -- SVR
C = 4
EPSILON = 0.1
#CACHE_SIZE = 3500
TOL = 1e-3

# -- KNR
N_NEIGHBORS = 7

# -- Model importance
IMPORTANCE = 0.5 # Out of 1.0 where 1.0 is 100% SVR and 0% KNR importance

#### Create Support Vector Regressor

In [ ]:
def create_svr(feature, fold_index, X_train_val, Y_train_val, X_test_val, Y_test_val):
    svr = SVR(C=[10 if feature=='age' else C][0], epsilon=EPSILON, verbose=True, tol=TOL) #, cache_size=CACHE_SIZE
    
    svr.fit(X_train_val, Y_train_val[feature])
    print('<<{} Fold SVR>> R2 training error: '.format(fold_index+1), svr.score(X_train_val, Y_train_val[feature])) # training error
    
    val_pred_svr = svr.predict(X_test_val)
    print('<<{} Fold SVR>> MSE validation error: '.format(fold_index+1), mean_squared_error(val_pred_svr, Y_test_val[feature])) # validation error
    
    test_pred_svr = svr.predict(X_test.drop("Id", axis=1))
    preds_svr[feature][fold_index] = test_pred_svr

#### Create K Neighbors Regressor

In [ ]:
def create_knr(feature, fold_index, X_train_val, Y_train_val, X_test_val, Y_test_val):
    knr = KNeighborsRegressor(n_neighbors=[9 if feature=='age' else N_NEIGHBORS][0], verbose=True)
    
    knr.fit(X_train_val, Y_train_val[feature])
    print('<<{} Fold KNR>> R2 training error: '.format(fold_index+1), knr.score(X_train_val, Y_train_val[feature])) # training error
    
    val_pred_knr = knr.predict(X_test_val)
    print('<<{} Fold KNR>> MSE validation error: '.format(fold_index+1), mean_squared_error(val_pred_knr.iloc[:,0], Y_test_val[feature])) # validation error
    
    test_pred_knr = knr.predict(X_test.drop("Id", axis=1))
    preds_knr[feature][fold_index] = test_pred_knr.iloc[:,0]

#### Define Validation parameters

In [ ]:
kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)

### Evaluation

In [ ]:
preds_svr = {}
preds_knr = {}

In [ ]:
for feature in Y_train.drop("Id", axis=1).columns:
    preds_svr[feature] = {}
    preds_knr[feature] = {}

    print("getting {}".format(feature))

    for fold_index, (train_index, val_index) in enumerate(kf.split(X_train, Y_train)):
        X_train_val, X_test_val = X_train.drop("Id", axis=1).iloc[train_index], X_train.drop("Id", axis=1).iloc[val_index]
        Y_train_val, Y_test_val = Y_train.drop("Id", axis=1).iloc[train_index], Y_train.drop("Id", axis=1).iloc[val_index]

        # -- SVR 
        create_svr(feature, fold_index, X_train_val, Y_train_val, X_test_val, Y_test_val)

        # -- KNR
        create_knr(feature, fold_index, X_train_val, Y_train_val, X_test_val, Y_test_val)

### Deployment

#### Results aggregation

In [ ]:
preds = cudf.DataFrame()
for feature in Y_train.drop("Id", axis=1).columns:
    preds.add_column(feature, ((IMPORTANCE * cudf.DataFrame(preds_svr[feature]) + (1 - IMPORTANCE) * cudf.DataFrame(preds_knr[feature])).sum(axis=1)) / 5)

#### Put values into submission

In [ ]:
submission = cudf.DataFrame(sample_submission)
submission['Predicted'] = preds.stack().to_frame()[0].values
submission

In [ ]:
submission.to_csv("submission.csv", index=False)

### Feedback

TODO:<br/>
* make hyperparameter tuning for svr and knr